In [1]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

!pip install wget

In [4]:
import wget
import os

print('Downloading dataset...')

# The URL for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

100% [........................................................] 255330 / 255330

# unzip

!unzip cola_public_1.1.zip

In [17]:
ls cola_public/tokenized

in_domain_dev.tsv      in_domain_train.tsv    out_of_domain_dev.tsv


In [10]:
import pandas as pd

#raw dataset

# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.head()

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.


In [11]:
df.label.value_counts()

1    6023
0    2528
Name: label, dtype: int64

In [13]:
df.sentence[10]

'The critics laughed the play off the stage.'

In [19]:
import pandas as pd

#raw dataset

# Load the dataset into a pandas dataframe.
df_tok = pd.read_csv("./cola_public/tokenized/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df_tok.shape[0]))

# Display 10 random rows from the data.
df_tok.head()

Number of training sentences: 8,551



,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"our friends wo n't buy this analysis , let alo..."
1,gj04,1,NaN,one more pseudo generalization and i 'm giving...
2,gj04,1,NaN,one more pseudo generalization or i 'm giving ...
3,gj04,1,NaN,"the more we study verbs , the crazier they get ."
4,gj04,1,NaN,day by day the facts are getting murkier .


In [21]:
df_tok.sample(10).sentence

5005                         it is eager to please john .
6689    sarah devoured the cakes in the kitchen last n...
5923    i feel it is unfortunate that television is so...
6936            what she did was e-mail all her friends .
2048                    sam gave the ball off the shelf .
4667             they widely believed that john was ill .
6532                          who do you regret you saw ?
3895                     john placed behind the counter .
7026                kim alienates cats and beat his dog .
1994    tonight , what bob cooked yesterday still tast...
Name: sentence, dtype: object

In [22]:
df_tok.sentence[1994]

'tonight , what bob cooked yesterday still tastes good .'

In [23]:
df.sentence[1994]

'Tonight, what Bob cooked yesterday still tastes good.'

In [28]:
#Stop words present in the library

from nltk.corpus import stopwords
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
#print(stopwords.words('english'))

#library that contains punctuation
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
stopwords = stopwords.words("english")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in stopwords\
              and token != " " \
              and token.strip() not in string.punctuation]
    
    text = " ".join(tokens)
    
    return text



In [32]:
preprocess_text('I was singing in the rain, just flowers growing in the garden')

'singing rain flowers growing garden'

In [34]:
df.sentence[10]

'The critics laughed the play off the stage.'

In [37]:
df_tok.head(2)

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"our friends wo n't buy this analysis , let alo..."
1,gj04,1,NaN,one more pseudo generalization and i 'm giving...


In [55]:
X = df_tok.sentence.apply(lambda x: preprocess_text(x))
y = df_tok.label

X = pd.DataFrame(X)
X.columns = ['text']

In [56]:
X.head(3)

,text
0,friends wo n buy analysis let alone next one p...
1,one pseudo generalization giving
2,one pseudo generalization giving


In [57]:
X.shape, y.shape

((8551, 1), (8551,))

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X.text, y, test_size=0.33, random_state=42)

In [72]:
vectoriser = TfidfVectorizer()

X_train = vectoriser.fit_transform(X_train)
# Convert sparse matrix to dataframe
#X_train = pd.DataFrame.sparse.from_spmatrix(X_train)

# Transform to feature matrix
X_test = vectoriser.transform(X_test)
# Convert sparse matrix to dataframe
#X_test = pd.DataFrame.sparse.from_spmatrix(X_test)
# Add column names to make it more readible


In [73]:
X_train.shape, X_test.shape

((5729, 4523), (2822, 4523))

In [75]:
lg = LogisticRegression(max_iter=10000)

In [76]:
lg.fit(X_train, y_train) # сошлось

LogisticRegression(max_iter=10000)

In [77]:
# лемматизация

print(classification_report(y_test, lg.predict(X_test))) #качество на тесте

              precision    recall  f1-score   support

           0       0.39      0.05      0.09       805
           1       0.72      0.97      0.83      2017

    accuracy                           0.71      2822
   macro avg       0.56      0.51      0.46      2822
weighted avg       0.63      0.71      0.62      2822



In [ ]:
как мы видим - качество не очень хорошее

In [ ]:
Попробуем настроить tf-idf без препроцессинга

In [78]:
X = df_tok.sentence
y = df_tok.label

X = pd.DataFrame(X)
X.columns = ['text']

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X.text, y, test_size=0.33, random_state=42)

In [96]:
vectoriser = TfidfVectorizer()

X_train = vectoriser.fit_transform(X_train)
# Convert sparse matrix to dataframe
#X_train = pd.DataFrame.sparse.from_spmatrix(X_train)

# Transform to feature matrix
X_test = vectoriser.transform(X_test)
# Convert sparse matrix to dataframe
#X_test = pd.DataFrame.sparse.from_spmatrix(X_test)
# Add column names to make it more readible



In [97]:
X_train.shape, X_test.shape

((5729, 4642), (2822, 4642))

In [98]:
lg = LogisticRegression(max_iter=10000)

lg.fit(X_train, y_train) # сошлось

LogisticRegression(max_iter=10000)

In [99]:
# лемматизация

print(classification_report(y_test, lg.predict(X_test))) #качество на тесте

              precision    recall  f1-score   support

           0       0.43      0.06      0.10       805
           1       0.72      0.97      0.83      2017

    accuracy                           0.71      2822
   macro avg       0.58      0.51      0.46      2822
weighted avg       0.64      0.71      0.62      2822



In [ ]:
качество практически не изменилось

In [ ]:
Оценим качество на валидационном датасете: 

In [86]:
test_df = pd.read_csv("./cola_public/tokenized/in_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])



In [87]:
test_df.head()

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,the sailors rode the breeze clear of the rocks .
1,gj04,1,NaN,the weights made the rope stretch over the pul...
2,gj04,1,NaN,the mechanical doll wriggled itself loose .
3,cj99,1,NaN,"if you had eaten more , you would want less ."
4,cj99,0,*,"as you eat the most , you want the least ."


In [88]:
test_df.shape # есть размеченные 527 валидационных примера

(527, 4)

In [91]:
X_holout = test_df.sentence
y_holdout = test_df.label

In [90]:
test_df.label.value_counts(1)

1    0.6926
0    0.3074
Name: label, dtype: float64

In [93]:
vectoriser.

TfidfVectorizer()

In [100]:
X_holout = vectoriser.transform(X_holout)


In [101]:
X_holout.shape

(527, 4642)

In [102]:
# лемматизация

print(classification_report(y_holdout, lg.predict(X_holout))) #качество на тесте

              precision    recall  f1-score   support

           0       0.44      0.07      0.12       162
           1       0.70      0.96      0.81       365

    accuracy                           0.69       527
   macro avg       0.57      0.51      0.46       527
weighted avg       0.62      0.69      0.60       527



In [ ]:
На основе валидационной выборки  мы видим, что качество такое же как на обучении

In [105]:
test_df.label_notes.unique()

array([nan, '*', '?*'], dtype=object)

In [106]:
df_tok.label_notes.unique()

array([nan, '*', '*?', '??', '?*'], dtype=object)

In [111]:
for el in df_tok.sentence.head():
    print(el)
    print()

our friends wo n't buy this analysis , let alone the next one we propose .

one more pseudo generalization and i 'm giving up .

one more pseudo generalization or i 'm giving up .

the more we study verbs , the crazier they get .

day by day the facts are getting murkier .



In [112]:
for el in df_tok.sentence.tail():
    print(el)
    print()

poseidon appears to own a dragon

digitize is my happiest memory

it is easy to slay the gorgon .

i had the strangest feeling that i knew you .

what all did you get for christmas ?



In [113]:
for el in df.sentence.tail():
    print(el)
    print()

Poseidon appears to own a dragon

Digitize is my happiest memory

It is easy to slay the Gorgon.

I had the strangest feeling that I knew you.

What all did you get for Christmas?



In [114]:
from sklearn.metrics import matthews_corrcoef


In [115]:
# Calculate the MCC
mcc = matthews_corrcoef(y_holdout, lg.predict(X_holout))

print('Total MCC: %.3f' % mcc)

Total MCC: 0.064


In [116]:
y_holdout.shape, lg.predict(X_holout).shape

((527,), (527,))

In [121]:
y_holdout

0      1
1      1
2      1
3      1
4      0
      ..
522    0
523    1
524    0
525    1
526    0
Name: label, Length: 527, dtype: int64

In [122]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [134]:
import numpy as np

In [138]:
print('accuracy', np.round(accuracy_score(y_holdout, lg.predict(X_holout)),3)) #качество на тесте
print('f1', np.round(f1_score(y_holdout, lg.predict(X_holout)),2)) #качество на тесте
print('roc_auc', np.round(roc_auc_score(y_holdout, lg.predict_proba(X_holout)[:,1]),3)) #качество на тесте
print('mcc',  np.round(matthews_corrcoef(y_holdout, lg.predict(X_holout)),3))

accuracy 0.687
f1 0.81
roc_auc 0.604
mcc 0.064
